Used ColQwen as retriever and used colQwen2 and MiniCPM as Generators

In [1]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git byaldi  accelerate  qwen_vl_utils pdf2image huggingface-hub
!sudo apt-get update
!sudo apt-get install -y poppler-utils
!pip install flash-attn
!pip install sentencepiece==0.1.99 bitsandbytes decord Pillow==10.1.0

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5zxovoxm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5zxovoxm
  Resolved https://github.com/huggingface/transformers.git to commit a3d69a8994d673899608a7c17fbf4f953f50474e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Byaldi-0.0.7-py3-none-any.whl.metadata (20 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached qwen_vl_utils-0.0.8-py3-none-any.whl.metadata (3.6 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached colpali_engine-0.3.4-py3-none-any.whl.metadata (21 kB)
  Using cached ml_

In [ ]:
#colab 
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, AutoModel
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path

In [3]:
pdf_path = "/home/Redacted.pdf"

In [4]:


RAG = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0")

RAG.index(
    input_path=pdf_path, # The path to your documents
    index_name="index", # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=False, # Whether the index should store the base64 encoded documents.
    # doc_ids=[], # Optionally, you can specify a list of document IDs. They must be integers and match the number of documents you're passing. Otherwise, doc_ids will be automatically created.
    # metadata=[], # Optionally, you can specify a list of metadata for each document. They must be a list of dictionaries, with the same length as the number of documents you're passing.
    overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 to index.
Added page 26 of document 0 to index.
Added page 27 of docu

{0: '/home/Redacted.pdf'}

In [ ]:
qwen_model = Qwen2VLForConditionalGeneration.from_pretrained(
                "Qwen/Qwen2-VL-7B-Instruct", 
                torch_dtype=torch.bfloat16,  
                attn_implementation="flash_attention_2", 
                device_map="cuda"
                )

qwen_processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", trust_remote_code=True)

In [ ]:
#MiniCPM
cpm_model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
                                # attn_implementation='sdpa', 
                                torch_dtype=torch.bfloat16, 
                                use_auth_token = ""
                                ) # sdpa or flash_attention_2, no eager
cpm_model = cpm_model.eval().cuda()
cpm_model_tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', 
                                            trust_remote_code=True,
                                            use_auth_token=""
                                            )
cpm_model_processor = AutoProcessor.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True, use_auth_token = "")

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/processing_auto.py:230: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:521: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [49]:
query = input("Enter your query: ")
results = RAG.search(query, k=5)
print(results)

[{'doc_id': 0, 'page_num': 27, 'score': 21.0, 'metadata': {}, 'base64': None}, {'doc_id': 0, 'page_num': 12, 'score': 18.625, 'metadata': {}, 'base64': None}, {'doc_id': 0, 'page_num': 15, 'score': 17.5, 'metadata': {}, 'base64': None}, {'doc_id': 0, 'page_num': 10, 'score': 17.375, 'metadata': {}, 'base64': None}, {'doc_id': 0, 'page_num': 46, 'score': 17.0, 'metadata': {}, 'base64': None}]


In [9]:
images = convert_from_path(pdf_path)

In [50]:

image_indexes = [pages['page_num']-1 for pages in results]
image_indexes

[26, 11, 14, 9, 45]

In [23]:
def generate_colpali_template(image_indexes):
    content = []
    for i in image_indexes:
        content.append({"type":"image", "image":images[i]})
        break   #breaking because more images cause out of memmory error
    return content

content = generate_colpali_template(image_indexes)
content.append({"type": "text", "text": query})

messages = [
    {
        "role": "user",
        "content": content
    }
]



text = qwen_processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = qwen_processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


generated_ids = qwen_model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = qwen_processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The blood pressure (BP) of the patient on 20-10-22 is 106/61.']


In [51]:

def generate_mini_cpm_content(image_indexes, query):
  content = []
  for i in image_indexes:
    content.append(images[i])
  content.append(query)
  print(content)
  return content

question = f"""Based on the images provided, please provide a detailed and comprehensive answer to the following question. Include thorough explanations, relevant details, and any necessary examples to support your response.

Question: {query}
"""
content =  generate_mini_cpm_content(image_indexes, query)
msgs = [{'role': 'user', 'content': content}]

answer = cpm_model.chat(
    image=None,
    msgs=msgs,
    tokenizer=cpm_model_tokenizer,
    processor = cpm_model_processor
)
print(f"{query} : {answer}")

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7F2FEC7C2FB0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7F3020C82B60>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7F3020C829E0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7F3020C83340>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7F2FEC336680>, 'provide the past diseses of the patient']
provide the past diseses of the patient : The patient denied any previous accidents/injuries, prior surgeries, past illnesses, current medications, allergies to medication, and psychological disorders.
